**Tutorial NER dengan Metode CRF**

Sumber: 

https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2

https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html



Install library CRF

In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 5.3MB/s 


Import library yang diperlukan


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report

Baca dataset. Link file dataset: https://drive.google.com/file/d/17Hm26nL6EQ8aDrr_Abz3KqUkhnxa_Uoi/view?usp=sharing 

Jangan lupa sesuaikan path!

In [ ]:
df = pd.read_csv('drive/My Drive/test_code_2020/NLP/pekan_13/data/ner_100000_dataset.csv', encoding = "ISO-8859-1")

df.head()

,Unnamed: 0,Sentence #,Word,POS,Tag
0,0,Sentence: 1,Thousands,NNS,O
1,1,NaN,of,IN,O
2,2,NaN,demonstrators,NNS,O
3,3,NaN,have,VBP,O
4,4,NaN,marched,VBN,O


Preprocess, update NaN pada kolom Sentence # dengan nomor kalimat yang ada di baris terakhir sebelumnya.

In [ ]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(4544, 10922, 17)

Cek persebaran tag. Dapat dilihat jika persebarannya tidak seimbang (imbalanced)

In [ ]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,75
1,B-eve,53
2,B-geo,3303
3,B-gpe,1740
4,B-nat,30
5,B-org,1876
6,B-per,1668
7,B-tim,1823
8,I-art,43
9,I-eve,47


Class untuk mengambil data per sentence

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

Fungsi untuk ekstraksi fitur

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]
    
def sent2tokens(sent):
    return [token for token, postag, label in sent]

Kelas/label tag, diproses supaya nanti label O tidak diperhitungkan untuk evaluasi

In [ ]:
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim']

Ekstraksi fitur dan split data menjadi data train dan data test

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

Cek salah satu instance di data train

In [ ]:
print(X_train[0])
print(y_train[0])

[{'bias': 1.0, 'word.lower()': 'a', 'word[-3:]': 'A', 'word[-2:]': 'A', 'word.isupper()': True, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'DT', 'postag[:2]': 'DT', 'BOS': True, '+1:word.lower()': 'second', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'JJ', '+1:postag[:2]': 'JJ'}, {'bias': 1.0, 'word.lower()': 'second', 'word[-3:]': 'ond', 'word[-2:]': 'nd', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'JJ', 'postag[:2]': 'JJ', '-1:word.lower()': 'a', '-1:word.istitle()': True, '-1:word.isupper()': True, '-1:postag': 'DT', '-1:postag[:2]': 'DT', '+1:word.lower()': 'nominee', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'NN', '+1:postag[:2]': 'NN'}, {'bias': 1.0, 'word.lower()': 'nominee', 'word[-3:]': 'nee', 'word[-2:]': 'ee', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NN', 'postag[:2]': 'NN', '-1:word.lower()': 'second', '-1:word.istitle()'

Pelatihan dengan metode CRF

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

Prediksi label data test.

Print metriks evaluasi.

In [ ]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

       B-art       1.00      0.03      0.07        29
       B-eve       0.86      0.25      0.39        24
       B-geo       0.75      0.88      0.81      1043
       B-gpe       0.89      0.78      0.83       588
       B-nat       0.67      0.20      0.31        10
       B-org       0.75      0.64      0.69       649
       B-per       0.81      0.81      0.81       546
       B-tim       0.90      0.85      0.87       589
       I-art       0.00      0.00      0.00         7
       I-eve       0.57      0.22      0.32        18
       I-geo       0.71      0.71      0.71       204
       I-gpe       0.47      0.53      0.50        17
       I-nat       1.00      0.50      0.67         2
       I-org       0.78      0.73      0.76       545
       I-per       0.80      0.90      0.85       574
       I-tim       0.79      0.68      0.73       185

   micro avg       0.80      0.78      0.79      5030
   macro avg       0.73   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
